In [37]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

In [43]:
HOME = os.getcwd()
IMAGE_DATA_PATH = f"{HOME}/Data/Weather_Dataset"
IMAGE_TENSOR_PATH = f"{HOME}/Data/Image_Tensors"

def load_jpg_to_tensor(image_filepath):
    """
    Reads a JPG image and converts it to a TensorFlow tensor.

    Args:
        image_path (str): Path to the JPG image file, relative to IMAGE_DATA_PATH

    Returns:
        A TensorFlow tensor representing the image.
    """
    # Read the image file
    absolute_image_filepath = os.sep.join((IMAGE_DATA_PATH, image_filepath))
    image_string = tf.io.read_file(absolute_image_filepath)
    
    # Decode the JPG image to a tensor
    image_tensor = tf.image.decode_jpeg(image_string, channels=3) 
    
    return image_tensor

def serialize_image_tensor(image_filepath):
    """
    Converts image to tensor, serializes tensor, and saves it to a binary file.

    Args:
        image_filenpath (str): Path to the JPG image file, relative to IMAGE_DATA_PATH
    """
    tensor = load_jpg_to_tensor(image_filepath)
    serialized_tensor = tf.io.serialize_tensor(tensor)

    image_filepath_no_ext = image_filepath.split(".")[0]
    serialized_tensor_filepath = os.sep.join((IMAGE_TENSOR_PATH, f"{image_filepath_no_ext}.bin"))

    directory = os.path.dirname(serialized_tensor_filepath) 

    # Ensure the directory exists
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

    with open(serialized_tensor_filepath, "wb") as f:
        f.write(serialized_tensor.numpy())

    return None

def deserialize_image_tensor(image_filepath):
    """
    Reads serialized tensor binary file associated with image,
    deserializes the tensor, and returns it.

    Args:
        image_filepath (str): Path to the JPG image file, relative to IMAGE_DATA_PATH

    Returns:
        tf.Tensor: Original Image Tensor
    """
    image_filepath_no_ext = image_filepath.split(".")[0]
    serialized_tensor_filepath = os.sep.join((IMAGE_TENSOR_PATH, f"{image_filepath_no_ext}.bin"))
    with open(serialized_tensor_filepath, "rb") as f:
        deserialized_tensor = tf.io.parse_tensor(f.read(), out_type=tf.uint8)
    return deserialized_tensor

### Verify serialization produces equivalent tensors

In [ ]:
import tensorflow as tf

image_filepath = "dew/2208.jpg"

# Create a sample tensor
original_tensor = load_jpg_to_tensor(image_filepath)

# Serialize the tensor using tf.io.serialize_tensor
serialize_image_tensor(image_filepath)

# To deserialize the tensor back to its original form, use tf.io.parse_tensor
deserialized_image_tensor = deserialize_image_tensor(image_filepath)    

# Verify that the deserialized tensor is the same as the original tensor
print(tf.reduce_all(tf.equal(original_tensor, deserialized_image_tensor)))

tf.Tensor(True, shape=(), dtype=bool)
